In [1]:
from datasets import load_dataset

dataset_files = {
    "train": ["/root/workspace/my_reserch/data/chABSA-dataset-tsv/train.csv"],
    "validation": ["/root/workspace/my_reserch/data/chABSA-dataset-tsv/validation.csv"],
    "test": ["/root/workspace/my_reserch/data/chABSA-dataset-tsv/test.csv"],
}
chABSA_dataset = load_dataset("csv", data_files=dataset_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e01a528f5ff8b78d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


/usr/local/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
import pprint
print(chABSA_dataset)
type(chABSA_dataset["train"])

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1688
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 562
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 563
    })
})


datasets.arrow_dataset.Dataset

In [3]:
from transformers import AutoTokenizer
model_ckpt = "nlp-waseda/roberta-base-japanese-with-auto-jumanpp"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [4]:
import torch.nn as nn
from transformers import AutoModel
from transformers.modeling_outputs import ModelOutput

class RobertaWithLstmForClassification(nn.Module):
    def __init__(self, pretrained_model, num_categories, loss_function=None):
        super().__init__()
        self.robert = pretrained_model
        self.hidden_size = self.robert.config.hidden_size
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, num_categories)
        self.loss_function = loss_function
    
    def forward(self,
                input_ids,
                attention_mask=None,
                position_ids=None,
                token_type_ids=None,
                output_attentions=False,
                output_hidden_states=False,
                labels=None):
        
        outputs = self.robert(input_ids,
                            attention_mask=attention_mask,
                            position_ids=position_ids,
                            token_type_ids=token_type_ids,
                            output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states)
        
        #state = outputs.last_hidden_state[:, 0, :]
        #state = self.lstm(state, None)
        #state = self.linear(state)
        
        out, _ = self.lstm(outputs['last_hidden_state'], None)
        sequence_output = out[:, -1, :]
        logits = self.linear(sequence_output)
        
        loss=None
        if labels is not None and self.loss_function is not None:
            loss = self.loss_function(logits, labels)
        
        attentions=None
        if output_attentions:
            attentions=outputs.attentions
        
        hidden_states=None
        if output_hidden_states:
            hidden_states=outputs.hidden_states
        
        return ModelOutput(
            logits=logits,
            loss=loss,
            last_hidden_state=outputs.last_hidden_state,
            attentions=attentions,
            hidden_states=hidden_states
        )

In [5]:
from torch import nn
from transformers import RobertaPreTrainedModel, RobertaModel
from transformers.modeling_outputs import SequenceClassifierOutput

class CustomModelForSequenceClassification(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = num_labels
        self.config = config

        # 系列分類の場合、[CLS]トークンのみの隠れ状態で十分なのでadd_pooling_layer=TrueでOK
        self.roberta = RobertaModel(config)

        #LSTMの用意
        self.lstm = nn.LSTM(config.hidden_size, config.hidden_size, batch_first=True)

        # 分類ヘッドの用意
        #self.dropout = nn.Dropout(config.hidden_dropout_prob)

        # ヘッドのlinear層について、入力の隠れ状態数にsource分の1を加算
        # self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        # Initialize weights and apply final processing
        # 初期化(Body部は事前学習済みの重みをロード、ヘッドはランダム初期化される)
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None\
        , labels=None, source=None, **kwargs):

        # robertaの隠れ状態を取得
        outputs = self.roberta(
            input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids, **kwargs
        )

        out, _ = self.lstm(outputs['last_hidden_state'], None)
        sequence_output = out[:, -1, :]
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [6]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

def tokenize(batch):
    tokenizer = AutoTokenizer.from_pretrained("nlp-waseda/roberta-base-japanese-with-auto-jumanpp")
    return tokenizer(batch["text"], padding=True, truncation=True)

In [7]:
chABSA_dataset["train"]

Dataset({
    features: ['text', 'labels'],
    num_rows: 1688
})

In [8]:
chABSA_dataset_encoded = chABSA_dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/1688 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/562 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/563 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
import torch
num_labels = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

loss_fct = nn.CrossEntropyLoss()
model_ckpt = "nlp-waseda/roberta-base-japanese-with-auto-jumanpp"
#model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))
#model = (RobertaWithLstmForClassification(pretrained_model, num_labels, loss_fct).to(device))
model = (CustomModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))
tokenizer = AutoTokenizer.from_pretrained("nlp-waseda/roberta-base-japanese-with-auto-jumanpp")


batch_size = 32
logging_steps = len(chABSA_dataset_encoded["train"]) // batch_size
output_dir = "/content/drive/MyDrive/Colab Notebooks/robarta-lstm"
training_args = TrainingArguments(output_dir=output_dir,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error")

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics, 
                  train_dataset=chABSA_dataset_encoded["train"], 
                  eval_dataset=chABSA_dataset_encoded["validation"],
                  tokenizer=tokenizer)

trainer.train()

Some weights of the model checkpoint at nlp-waseda/roberta-base-japanese-with-auto-jumanpp were not used when initializing CustomModelForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CustomModelForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomModelForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomModelForSequenceClassification were not initialized from the model checkpoint at nlp-waseda/roberta-base-japanese-with-auto-jumanpp and are newly initialized: ['lstm.weight_ih_l0', 'classifie

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED